<a href="https://colab.research.google.com/github/JayThibs/ai-safety-scrape/blob/main/scrape_ai_alignment_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting Contents from AI Alignment Resources

This notebook is used to scrape the contents of AI Alignment resources.

# Notebook Setup

## Installations

In [13]:
!python --version

Python 3.7.12


In [14]:
%env PYTHONPATH=

env: PYTHONPATH=


In [15]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-py37_4.11.0-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
Unpacking payload ...
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - _openmp_mutex==4.5=1_gnu
    - brotlipy==0.7.0=py37h27cfd23_1003
    - ca-certificates==2021.10.26=h06a4308_2
    - certifi==2021.10.8=py37h06a4308_2
    - cffi==1.15.0=py37hd667e15_1
    - charset-normalizer==2.0.4=pyhd3eb1b0_0
    - conda-content-trust==0.1.1=pyhd3eb1b0_0
    - conda-package-handling==1.7.3=py37h27cfd23_1
    - conda==4.11.0=py37h06a4308_0
    - cryptography==36.0.0=py37h9ce1e76_0
    - idna==3.3=pyhd3eb1b0_0
    - ld_impl_linux-64==2.35.1=h7274673_9
    - libffi==3.3=he6710b0_2
    - libgcc-ng==9.3.0=h5101ec6_17
    - libgomp==9.3.0=h5101ec6_17
    - libstdcxx-ng==9.3.0=hd4cf53a_17
    - ncurses==6.3=h7f8727e_2
    - openssl==1.1.1m=h7f8727e_0
    - pip==21.2.2=py37h06a4308_0
    - pycosat==0.6.3=py37h27cfd23_0
    - pycparser==2.21=pyhd3eb1b0_0
    - pyopenssl==21.0.0=py

--2022-03-11 02:31:39--  https://repo.continuum.io/miniconda/Miniconda3-py37_4.11.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-py37_4.11.0-Linux-x86_64.sh [following]
--2022-03-11 02:31:39--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.11.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103730670 (99M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.11.0-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 65.8M 2s
    50K .......... .......... .......... 

In [16]:
%%bash

conda install --channel defaults conda python=3.7 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - conda
    - python=3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.2.1   |       h06a4308_0         122 KB
    ------------------------------------------------------------
                                           Total:         122 KB

The following packages will be UPDATED:

  ca-certificates                     2021.10.26-h06a4308_2 --> 2022.2.1-h06a4308_0



ca-certificates-2022 | 122 KB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local


The following packages will be downloaded:

    package                    |            build
    ---

In [17]:
import sys

_ = (sys.path.append("/usr/local/lib/python3.7/site-packages"))

In [18]:
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/usr/local/lib/python3.7/site-packages']

In [19]:
!conda install -c conda-forge pypandoc --yes

Solving environment: | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pypandoc


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2021.10.8          |   py37h89c1867_1         145 KB  conda-forge
    conda-4.11.0               |   py37h89c1867_2        16.9 MB  conda-forge
    pandoc-2.17.1.1            |       ha770c72_0        11.9 MB  conda-forge
    pypandoc-1.7.2             |     pyhd8ed1ab_0          26 KB  conda-forge
    python_abi-3.7             |          2_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        29.1 MB

The following NEW packages will be INSTALLED:

  pandoc             conda-forge/linux-64

In [20]:
!conda install -c anaconda pip --yes

Solving environment: \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pip


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.10.14 |                0         128 KB  anaconda
    certifi-2020.6.20          |           py37_0         159 KB  anaconda
    pip-20.2.4                 |           py37_0         2.0 MB  anaconda
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2021.10.~ --> anaconda::ca-certificates-2020.10.14-0
  certifi            conda-forge::certifi-2021.10.8-py37h8~ --> anaconda::certifi-2020.6.20-py37_0
  pip                  pkgs/main::pip-21.2.2-py37h06a4308_0 --> anaconda::pip-20.

In [21]:
!pip install --upgrade pandoc -q

     |████████████████████████████████| 117 kB 6.2 MB/s 
     |████████████████████████████████| 49 kB 3.7 MB/s 


In [22]:
!apt-get update && apt-get install -y libmagic-dev -q

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpa

In [23]:
!pip install tika arxiv lm-dataformat python-magic==0.4.15 chardet beautifulsoup4 -q

     |████████████████████████████████| 178 kB 7.1 MB/s 
     |████████████████████████████████| 97 kB 2.7 MB/s 
     |████████████████████████████████| 81 kB 7.4 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 2.5 MB 34.4 MB/s 
     |████████████████████████████████| 60 kB 4.5 MB/s 


## Imports

In [24]:
import os
import pandas as pd
from tika import parser
from google.colab import drive
from pathlib import Path
from urllib import request
from bs4 import BeautifulSoup
import arxiv
import pypandoc
import requests
import tarfile
import pickle
from urllib.error import HTTPError
import json

## Setting up Environment

In [25]:
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [27]:
CODE_DIR = Path('.') / 'code-projects/gpt-ai-safety'
RAW_DIR = Path('data/raw')
INTERIM_DIR = Path('data/interim')
PROCESSED_DIR = Path('data/processed')
TARS_DIR = RAW_DIR / 'tars'
LATEX_DIR = RAW_DIR / 'latex_files'
PDFS_DIR = RAW_DIR / 'pdfs'
PKLS_DIR = INTERIM_DIR / 'pkls'
EXTRACTED_TARS_DIR = INTERIM_DIR / 'extracted_tars'
MERGE_TEX_DIR = INTERIM_DIR / 'merge_latex_files'
PROCESSED_TXTS_DIR = PROCESSED_DIR / 'txts'
PROCESSED_JSONS_DIR = PROCESSED_DIR / 'jsons'

In [28]:
!mkdir -p {CODE_DIR}

In [29]:
%cd {CODE_DIR}

/content/drive/MyDrive/code-projects/gpt-ai-safety


In [30]:
!mkdir -p {RAW_DIR} {INTERIM_DIR} {PROCESSED_DIR} {TARS_DIR} {LATEX_DIR} {PDFS_DIR}
!mkdir -p {PKLS_DIR} {EXTRACTED_TARS_DIR} {MERGE_TEX_DIR} {PROCESSED_TXTS_DIR} {PROCESSED_JSONS_DIR}

In [39]:
if not os.path.exists('ai-alignment-papers.csv'):
    !git clone https://github.com/JayThibs/ai-safety-scrape
    !mv ai-safety-scrape/* .

Cloning into 'ai-safety-scrape'...
remote: Enumerating objects: 228, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 228 (delta 135), reused 102 (delta 45), pack-reused 0
Receiving objects: 100% (228/228), 2.10 MiB | 3.32 MiB/s, done.
Resolving deltas: 100% (135/135), done.


# Load and Explore Data

In [42]:
df = pd.read_csv('ai-alignment-papers.csv', index_col=0)
df.head()

,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,...,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body
0,XBZAPQFK,blogPost,2020.0,"Kokotajlo, Daniel",Three kinds of competitiveness,AI Impacts,NaN,NaN,NaN,https://aiimpacts.org/three-kinds-of-competiti...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HX9UZ5JP,journalArticle,2020.0,"Cihon, Peter; Maas, Matthijs M.; Kemp, Luke",Fragmentation and the Future: Investigating Ar...,Global Policy,NaN,1758-5899,10.1111/1758-5899.12890,https://onlinelibrary.wiley.com/doi/abs/10.111...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BQCZM53S,blogPost,2021.0,"Clarke, Sam; Martin, Samuel Dylan",Distinguishing AI takeover scenarios,AI Alignment Forum,NaN,NaN,NaN,https://www.alignmentforum.org/posts/qYzqDtoQa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JVMJ4RMM,journalArticle,2020.0,"Stray, Jonathan",Aligning AI Optimization to Community Well-Being,International Journal of Community Well-Being,NaN,"2524-5295, 2524-5309",10.1007/s42413-020-00086-3,http://link.springer.com/10.1007/s42413-020-00...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,W8F6VI9I,thesis,2020.0,"Shah, Rohin Monish",Extracting and Using Preference Information fr...,NaN,NaN,NaN,NaN,https://www.proquest.com/openview/da8bf63ef343...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df['Item Type'].unique()

array(['blogPost', 'journalArticle', 'thesis', 'conferencePaper',
       'manuscript', 'report', 'bookSection', 'magazineArticle', 'book',
       'webpage', 'encyclopediaArticle'], dtype=object)

In [44]:
item_nums = []
for item in df['Item Type'].unique():
    item_nums.append([item, len(df[df['Item Type'] == item])])

item_nums.sort(key=lambda x:x[1])
item_nums

[['encyclopediaArticle', 1],
 ['magazineArticle', 2],
 ['thesis', 3],
 ['book', 14],
 ['webpage', 16],
 ['bookSection', 51],
 ['report', 80],
 ['manuscript', 133],
 ['conferencePaper', 237],
 ['blogPost', 428],
 ['journalArticle', 747]]

In [45]:
df[df['Item Type'] == 'journalArticle'].head()

,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,...,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body
1,HX9UZ5JP,journalArticle,2020.0,"Cihon, Peter; Maas, Matthijs M.; Kemp, Luke",Fragmentation and the Future: Investigating Ar...,Global Policy,NaN,1758-5899,10.1111/1758-5899.12890,https://onlinelibrary.wiley.com/doi/abs/10.111...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JVMJ4RMM,journalArticle,2020.0,"Stray, Jonathan",Aligning AI Optimization to Community Well-Being,International Journal of Community Well-Being,NaN,"2524-5295, 2524-5309",10.1007/s42413-020-00086-3,http://link.springer.com/10.1007/s42413-020-00...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,TK5F29IU,journalArticle,2021.0,"Hayden, Benjamin; Niv, Yael",The case against economic values in the orbito...,Behavioral Neuroscience,NaN,NaN,10.1037/bne0000448,https://osf.io/7hgup,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NHWZIKZ2,journalArticle,2020.0,"Fernandes, Pedro; Santos, Francisco C.; Lopes,...",Norms for Beneficial A.I.: A Computational Ana...,AI Communications,NaN,"18758452, 09217126",10.3233/AIC-201502,http://arxiv.org/abs/1907.03843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,HDWGJGAP,journalArticle,2021.0,"Mingard, Chris; Valle-Pérez, Guillermo; Skalse...","Is SGD a Bayesian sampler? Well, almost",Journal of Machine Learning Research,NaN,NaN,NaN,http://arxiv.org/abs/2006.15191,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Extract text from AI Alignment Resources

## Downloading Arxiv Papers

In [59]:
df_arxiv = df[df['Url'].str.contains('arxiv') == True]
arxiv_paper_list = df_arxiv['Url'].values
original_len = len(arxiv_paper_list)
arxiv_paper_list = set(arxiv_paper_list)
removed_duplicates = original_len - len(arxiv_paper_list)
print('Original number of papers: ' + str(original_len))
print('Duplicates removed: ' + str(removed_duplicates))
print('Current number of papers: ' + str(len(arxiv_paper_list)))

Original number of papers: 850
Duplicates removed: 129
Current number of papers: 721


In [53]:
if os.path.exists(str(PKLS_DIR / 'arxiv_paper_tars_list.pkl')):
    with open(str(PKLS_DIR / 'arxiv_paper_tars_list.pkl'), 'rb') as f:
        arxiv_paper_tars = pickle.load(f)
else:
    arxiv_paper_tars = [0] * len(arxiv_paper_list)

def download_arxiv_paper_tars(papers=arxiv_paper_list, tars=arxiv_paper_tars, create_dict_only=False):
    incorrect_links_ids = []
    paper_dl_failures = []
    arxiv_dict = {}
    for i, (paper_link, filename) in enumerate(zip(papers, tars)):
        paper_id = '.'.join(filename.split('.')[:2])
        
        if os.path.exists(str(TARS_DIR / filename)) and create_dict_only==False:
            print('Already downloaded the ' + paper_id + " tar file.")
            continue

        try:
            paper_id = paper_link.split('/')[-1]
            paper = next(arxiv.Search(id_list=[paper_id]).results())
            arxiv_dict[paper.get_short_id()[:-2]] = {
                'source': 'arxiv',
                'source_filetype': 'latex',
                'converted_with': 'pandoc',
                "post_title": paper.title,
                "author": [str(x) for x in paper.authors],
                "date_published": str(paper.published),
                "data_last_modified": str(paper.updated),
                "url": str(paper.entry_id),
                "abstract": paper.summary,
                "author_comment": paper.comment,
                "journal_ref": paper.journal_ref,
                "doi": paper.doi,
                "primary_category": paper.primary_category,
                "categories": paper.categories}
            if create_dict_only:
                print('Added ' + paper.get_short_id()[:-2] + ' to json.')
                continue
            tar_filename = paper.entry_id.split('/')[-1] + '.tar.gz'
            tars[i] = tar_filename
        except:
            incorrect_links_ids.append([paper_link, paper_id])
            pass
        
        try:
            paper.download_source(dirpath=str(TARS_DIR), filename=filename)
            print('Downloaded paper: ' + paper_id)
        except HTTPError:
            print('Could not download paper: ' + paper_id)
            paper_dl_failures.append(paper_id)
            pass

    print(incorrect_links_ids), print(paper_dl_failures)

    with open(PROCESSED_JSONS_DIR / 'arxiv_dict.json', 'w') as fp:
        json.dump(arxiv_dict, fp)

    with open(PKLS_DIR / 'arxiv_paper_tars_list.pkl', 'wb') as f:
        pickle.dump(tars, f)

    with open(PKLS_DIR / 'incorrect_links_ids_list.pkl', 'wb') as f:
        pickle.dump(incorrect_links_ids, f)

    with open(PKLS_DIR / 'paper_dl_failures_list.pkl', 'wb') as f:
        pickle.dump(paper_dl_failures, f)


In [56]:
download_arxiv_paper_tars(create_dict_only=False)

Already downloaded the 2002.11328v3 tar file.
Already downloaded the 2010.11645v2 tar file.
Already downloaded the 2002.11708v1 tar file.
Already downloaded the 2012.10800v1 tar file.
Already downloaded the 2011.08512v1 tar file.
Already downloaded the 2002.11089v1 tar file.
Already downloaded the 2012.05876v2 tar file.
Already downloaded the 2010.14603v1 tar file.
Already downloaded the 2005.01643v3 tar file.
Already downloaded the 1907.03843v2 tar file.
Already downloaded the 2009.12612v2 tar file.
Already downloaded the 2007.08124v1 tar file.
Already downloaded the 2006.15191v2 tar file.
Already downloaded the 1909.13371v1 tar file.
Already downloaded the 2008.08076v2 tar file.
Already downloaded the 2009.00802v1 tar file.
Already downloaded the 2011.08541v1 tar file.
Already downloaded the 2009.08092v2 tar file.
Already downloaded the 2003.07305v1 tar file.
Already downloaded the 2004.09044v1 tar file.
Already downloaded the 1604.00289v3 tar file.
Already downloaded the 2007.02382v

## Extract LaTeX papers from arXiv

In [31]:
!gunzip files/*.tar.gz

In [ ]:
!python arxiv_extractor.py

  0% 0/280 [00:00<?, ?it/s]waiting for tar...
finished preload of 1606.05374v1.ta
finished preload of 1704.02882v2.ta
files/1704.02882v2.tar.gz
tmp/Figures/bothvaried_tex.tex
tmp/Figures/nll_pgf.tex
tmp/Figures/combined_system_flowchart.tex
tmp/Figures/DomainofAttraction.tex
True
tmp/Figures/drop_pgf.tex
True
tmp/Figures/bothfixed6_learning.tex
True
tmp/Figures/bothvaried6_learning.tex
True
tmp/Figures/gp_pgf.tex
True
tmp/Figures/bothfixed6_areas.tex
True
tmp/Figures/fig3tikz.tex
True
tmp/Figures/HalfCheetahIL.tex
True
tmp/Figures/bothfixed6_failures.tex
True
tmp/Figures/mse_pgf.tex
True
True
True
True
marking files/1704.02882v2.tar.gz as done
  0% 1/280 [00:01<08:43,  1.88s/it]files/1606.05374v1.tar.gz
  1% 2/280 [00:07<18:51,  4.07s/it]waiting for tar...
finished preload of 1609.03543v5.ta
finished preload of 1608.04112v6.ta
files/1609.03543v5.tar.gz
  1% 3/280 [00:08<13:19,  2.88s/it]finished preload of 1609.04994v3.ta
files/1608.04112v6.tar.gz
  1% 4/280 [00:09<08:49,  1.92s/it]fin

In [38]:
def extract_tex(tar_filename):
    tar = tarfile.open(str(TARS_DIR / tar_filename))
    tex_filename = None
    for member in tar.getmembers():
        if ".tex" in member.name:
            tex_filename = member.name
            tar.extract(member, LATEX_DIR)
    tar.close()
    return tex_filename

In [39]:
pickled_arxiv_list = str(PKLS_DIR / "arxiv_ids_and_names.pkl")

if not os.path.exists(pickled_arxiv_list):
    arxiv_ids_and_names = []
else:
    pickle.load(pickled_arxiv_list)

In [ ]:
arxiv_ids_and_names = []
pdf_ids = []
for paper_id in arxiv_paper_list[20:25]:
    paper_id_v1 = paper_id.split('/')[-1] + "v1"
    # print(f'paper_id: {paper_id_v1}')
    tar_filename = paper_id_v1 + ".tar.gz"
    # print(f'tar_filename: {tar_filename}')
    # download_arxiv_paper_tar(paper_id_v1, tar_filename)
    # print("downloaded")
    if not os.path.exists(str(LATEX_DIR / tar_filename)):
        tex_filename = extract_tex(tar_filename)
    if tex_filename:
        arxiv_ids_and_names.append([paper_id, tex_filename])
    else: 
        #Is pdf only, do the pdf-scraper tiki later
        pdf_ids.append(paper_id)

Paper Entry ID: http://arxiv.org/abs/2009.12612v1
Couldn't download paper.
Paper Entry ID: http://arxiv.org/abs/2007.08124v1
Couldn't download paper.


In [ ]:
with open('arxiv_ids_and_names.pkl', 'wb') as b:
    pickle.dump(arxiv_ids_and_names, b)

## Convert LaTeX or EPUB with Pandoc

In [ ]:
!python arxiv_extractor.py

## Extracting PDFs with Tika

In [ ]:
def tikaTextExtractor(file_path):
    """Extracts text from a PDF using tika."""
    print("Extracting text from file: " + file_path)
    parsed_tika = parser.from_file(file_path)
    return parsed_tika["content"]

txt = tikaTextExtractor(str(RAW_DIR / 'pdfs' / 'Superintelligence.pdf'))

In [ ]:
print(txt[0:1000])

In [ ]:
with open(PROCESSED_TXTS_DIR / 'Superintelligence.txt', 'w') as f:
    f.write(txt)

# Search all useful arxiv papers

In [ ]:
search = arxiv.Search(
    query="ai safety",
    max_results=10,
    sort_by = arxiv.SortCriterion.Relevance
)

In [ ]:
for result in search.results():
  print('Title: ', result.title, '\nDate: ',result.published , '\nId: ', result.entry_id, 
        '\nSummary: ',result.summary ,'\nURL: ', result.pdf_url, '\n\n')

Title:  Evolutionary Computation and AI Safety: Research Problems Impeding Routine and Safe Real-world Application of Evolution 
Date:  2019-06-24 19:26:10+00:00 
Id:  http://arxiv.org/abs/1906.10189v2 
Summary:  Recent developments in artificial intelligence and machine learning have
spurred interest in the growing field of AI safety, which studies how to
prevent human-harming accidents when deploying AI systems. This paper thus
explores the intersection of AI safety with evolutionary computation, to show
how safety issues arise in evolutionary computation and how understanding from
evolutionary computational and biological evolution can inform the broader
study of AI safety. 
URL:  http://arxiv.org/pdf/1906.10189v2 


Title:  AI Failures: A Review of Underlying Issues 
Date:  2020-07-18 15:31:29+00:00 
Id:  http://arxiv.org/abs/2008.04073v1 
Summary:  Instances of Artificial Intelligence (AI) systems failing to deliver
consistent, satisfactory performance are legion. We investigate w

# SOTAWHAT

In [ ]:
!git clone https://github.com/chiphuyen/sotawhat
%cd sotawhat
!pip install .
%cd ..

Cloning into 'sotawhat'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 92 (delta 0), reused 1 (delta 0), pack-reused 89
Unpacking objects: 100% (92/92), done.
/content/drive/MyDrive/code-projects/gpt-ai-safety/sotawhat
Processing /content/drive/MyDrive/code-projects/gpt-ai-safety/sotawhat
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 2.7 MB 5.1 MB/s 
  Created wheel for sotawhat: filename=sotawhat-0.0.1-py3-none-any.whl size=4623 sha256=5aef8419ceec258d0ead1cf1725627fcb3cf7f5d4dfae58b5c359b5929819a17
  St

In [ ]:
!sotawhat ai safety 10

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Impossibility Results in AI: A Survey (Mario Brcic - 1 September, 2021)
As such, these results serve as guidelines, reminders, and warnings to AI safety, AI policy, and governance researchers. We concluded that deductive impossibilities deny 100%-guarantees for security
Link: sp;[<a href=
Towards a framework for evaluating the safety, acceptability and efficacy of AI systems for health: an initial synthesis (Jessica Morley - 14 April, 2021)
There are numerous reasons for this, but a main contributory factor is the lack of internationally accepted, or formalised, regulatory standards to assess AI safety and impact and effectiveness. We do this by conducting a systematic search across Scopus, PubMed and Google Scholar to identify all the relevant literature published between January 1970 and November 2020 related to the evaluation of: output performance; efficacy; and real-world use 